In [12]:
import osmnx as ox
import pandas as pd
from shapely.geometry import Point

def extract_osm_features(lat, lon, radius=1000):
    """
    Extract features from OpenStreetMap around a specific coordinate.

    Parameters:
        lat (float): Latitude of the central point.
        lon (float): Longitude of the central point.
        radius (int): Radius (in meters) for the search area.

    Returns:
        DataFrame: A pandas DataFrame with extracted features.
    """
    # Define the central point and buffer area
    center_point = (lat, lon)

    # Extract data from OSM
    G = ox.graph_from_point(center_point, dist=radius, network_type='all')
    buildings = ox.features_from_point(center_point, tags={'building': True}, dist=radius)
    parks = ox.features_from_point(center_point, tags={'leisure': 'park'}, dist=radius)
    roads = ox.graph_to_gdfs(G, nodes=False, edges=True)
    low_emission_zones = ox.features_from_point(center_point, tags={'low_emission_zone': True}, dist=radius)


    # Calculate features
    data = []

    # Proximity to major roads
    if not roads.empty:
        major_roads = roads[roads['highway'].isin(['motorway', 'trunk', 'primary'])]
        distances_to_roads = major_roads.geometry.distance(Point(lon, lat))
        min_distance_to_major_road = distances_to_roads.min()
    else:
        min_distance_to_major_road = None

    # Green space coverage
    if not parks.empty:
        green_area = parks.geometry.area.sum()
    else:
        green_area = 0

    # Building density
    if not buildings.empty:
        building_count = len(buildings)
        avg_building_height = buildings['building:levels'].dropna().astype(float).mean() if 'building:levels' in buildings.columns else None
    else:
        building_count = 0
        avg_building_height = None

    # Append data
    data.append({
        'latitude': lat,
        'longitude': lon,
        'min_distance_to_major_road_m': min_distance_to_major_road,
        'green_area_m2': green_area,
        'building_count': building_count,
        'avg_building_height_levels': avg_building_height,
    })

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

In [14]:
def extract_osm_features(lat, lon, radius=1000):
    """
    Extract enhanced features from OpenStreetMap around a specific coordinate.

    Parameters:
        lat (float): Latitude of the central point.
        lon (float): Longitude of the central point.
        radius (int): Radius (in meters) for the search area.

    Returns:
        DataFrame: A pandas DataFrame with extracted features.
    """
    # Define the central point and buffer area
    center_point = (lat, lon)

    # Extract data from OSM
    G = ox.graph_from_point(center_point, dist=radius, network_type='all')
    buildings = ox.features_from_point(center_point, tags={'building': True}, dist=radius)
    parks = ox.features_from_point(center_point, tags={'leisure': 'park'}, dist=radius)
    amenities = ox.features_from_point(center_point, tags={'amenity': True}, dist=radius)
    water_bodies = ox.features_from_point(center_point, tags={'natural': 'water'}, dist=radius)
    roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

    # Calculate features
    data = []

    # Proximity to major roads
    if not roads.empty:
        major_roads = roads[roads['highway'].isin(['motorway', 'trunk', 'primary'])]
        distances_to_roads = major_roads.geometry.distance(Point(lon, lat))
        min_distance_to_major_road = distances_to_roads.min()
        intersection_density = len(roads[roads['junction'] == 'intersection'])
    else:
        min_distance_to_major_road = None
        intersection_density = 0

    # Green space coverage
    if not parks.empty:
        green_area = parks.geometry.area.sum()
    else:
        green_area = 0

    # Building density
    if not buildings.empty:
        building_count = len(buildings)
        avg_building_height = buildings['building:levels'].dropna().astype(float).mean() if 'building:levels' in buildings.columns else None
    else:
        building_count = 0
        avg_building_height = None

    # Water bodies proximity
    if not water_bodies.empty:
        distances_to_water = water_bodies.geometry.distance(Point(lon, lat))
        min_distance_to_water = distances_to_water.min()
    else:
        min_distance_to_water = None

    # Amenity count
    amenity_count = len(amenities)

    # Append data
    data.append({
        'latitude': lat,
        'longitude': lon,
        'min_distance_to_major_road_m': min_distance_to_major_road,
        'intersection_density': intersection_density,
        'green_area_m2': green_area,
        'building_count': building_count,
        'avg_building_height_levels': avg_building_height,
        'min_distance_to_water_m': min_distance_to_water,
        'amenity_count': amenity_count,
    })

    # Create DataFrame
    df = pd.DataFrame(data)
    return df


In [15]:
# Example usage
latitude = 40.416775  # Madrid city center latitude
longitude = -3.703790  # Madrid city center longitude
radius_in_meters = 1000

df_features = extract_osm_features(latitude, longitude, radius=radius_in_meters)

/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_30915/533685411.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances_to_roads = major_roads.geometry.distance(Point(lon, lat))
/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_30915/533685411.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  green_area = parks.geometry.area.sum()
/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_30915/533685411.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances_to_water = water_bodies.geometry.distance(Point(lon, lat))


In [16]:
df_features

,latitude,longitude,min_distance_to_major_road_m,intersection_density,green_area_m2,building_count,avg_building_height_levels,min_distance_to_water_m,amenity_count
0,40.416775,-3.70379,0.009172,0,0.000128,6114,4.942329,0.000115,4570
